# Cvičení 3.5

Tématem tohoto cvičení jsou přímé metody řešení soustav lineárních rovnic. Budeme se zabývat LU rozkladem s řádkovými permutacemi.

# LU rozklad s částečnou pivotizací

Algoritmus LU rozkladu s částečnou pivotizací lze popsat např. následujícím pseudokódem.

```
U = A, L = I, P = I
for k = 1 to m-1 do
    Select i >= k maximizing abs(U(i, k))
    Swap U(k, k:m) <--> U(i, k:m)
    Swap L(k, 1:k-1) <--> L(i, 1:k-1)
    Swap P(k, :) <--> P(i, :)
    for j = k+1 to m do
        L(j, k) = U(j, k)/U(k, k)
        U(j, k:m) = U(j, k:m) - L(j, k)U(k, k:m)
    end for
end for
```

**Pozor**: značení `k:m` zde znamená sloupce od `k` po `m` *včetně*.

Na základě tohoto pseudokódu doplňte chybějící části následují funkce.

In [1]:
# ÚKOL: Doplňte následující kód

import numpy as np

def select_i(U, k):
    m,n = np.shape(U)
    i = k
    max = abs(U[i][k])
    for j in range(k, m):
        max2 = abs(U[i][k])
        if max2 > max:
            max = max2
            i = j
    return i

def swap(a, b):
    return b, a

def lu_decomposition_p(A):
    m,n = np.shape(A)
    U = A.copy()
    ones = np.ones(m)
    L = np.diag(ones, 0)
    P = np.diag(ones, 0)

    for k in range(0, m-1):
        i = select_i(U, k)
        U[k][k:m], U[i][k:m] = swap(U[k][k:m], U[i][k:m])
        L[k][1:k], L[i][1:k] = swap(L[k][1:k], L[i][1:k])
        P[k][:], P[i][:] = swap(P[k][:], P[i][:])
        for j in range(k+1, m):
            L[j][k] = U[j][k]/U[k][k]
            U[j][k:m+1] = U[j][k:m+1] - L[j][k] * U[k][k:m+1]
    return L, U, P

    """
    Provádí LU dekompozici s částečnou pivotizací: PA = LU
    Použití: L, U, P = lu_decomposition_p(A)
    """


In [2]:
# Vytvoříme náhodnou matici 5x5 a zavoláme vaši funkci
A = np.random.rand(5, 5)
L, U, P = lu_decomposition_p(A)

In [3]:
# Otestujeme, že funkce vrací správný rozklad
np.set_printoptions(precision=3)
LU = L@U
print(LU)
print(P@A)

[[0.435 0.919 0.833 0.366 0.825]
 [0.4   0.822 0.262 0.182 0.951]
 [0.119 0.045 0.55  0.436 0.259]
 [0.43  0.954 0.719 0.652 0.625]
 [0.75  0.703 0.545 0.444 0.735]]
[[0.435 0.919 0.833 0.366 0.825]
 [0.4   0.822 0.262 0.182 0.951]
 [0.119 0.045 0.55  0.436 0.259]
 [0.43  0.954 0.719 0.652 0.625]
 [0.75  0.703 0.545 0.444 0.735]]


Známe-li rozklad matice $A$ ve tvaru
$$\mathsf{L}\mathsf{U} = \mathsf{P}\mathsf{A},$$
můžeme jej využít k řešení soustavy $\mathsf{A}\mathbf{x} = \mathbf{b}$. Soustavu upravíme na tvar 
$$\mathsf{P}\mathsf{A}\mathbf{x} = \mathsf{P}\mathbf{b}$$
a dosadíme
$$\mathsf{L}\mathsf{U}\mathbf{x}=\mathsf{P}\mathbf{b}.$$

V následující části využijte rozklad matice A a vaše metody `fsubst` a `bsubst` pro dopřednou a zpětnou substituci k vyřešení soustavy. 

In [4]:
# ÚKOL: Zkopírujte z minulého cvičení

def fsubst(L, b):
    m, n = np.shape(L)       
    x = np.zeros((n,1))         
    
    x[0][0] = b[0][0] / L[0, 0] 

    for i in range(1, m):
        suma = 0
        for j in range(0,i):                       
            suma += L[i][j]*x[j][0]                        
        x[i][0] = (b[i][0] - suma) / L[i][i]                           
    return x

In [5]:
# ÚKOL: Zkopírujte z minulého cvičení

def bsubst(U, b):
    m, n = np.shape(U)
    x = np.zeros(m)
    x[m-1] = b[m-1][0] / U[m-1][n-1]
    for i in range(m-2, -1, -1):
        suma = 0
        for j in range(i, m):
            suma += U[i][j]*x[j]
        x[i] = (b[i][0] - suma) / U[i][i]
    return x

In [6]:
# ÚKOL: Vytvořte funkci random_system, která bude mít na vstupu dimenzi m matice A a vrátí náhodnou
# matici o rozměrech m x m a náhodný vektor pravé strany b délky m

def random_system(m):
    A = np.random.rand(m,m)
    b = np.random.rand(m,1)
    return A, b

In [7]:
# Použijeme vaši metodu k vytvoření náhodné soustavy Ax=b o 5 neznámých
A, b = random_system(5)

In [8]:
# Rozložíme matici A pomocí vaší implementace LU rozkladu s pivotizací
L, U, P = lu_decomposition_p(A)

In [9]:
# ÚKOL: Vyřešte pomocí získaného rozkladu a dopředné a zpětné substituce soustavu Ax = b
#LUx = Pb

y = fsubst(L, P@b)

x = bsubst(U, y)
print(x)

[ 0.456  0.274  0.384 -0.392  0.719]


In [10]:
# ÚKOL: Porovnejte vaše řešení s výsledkem získaným pomocí numpy
x = np.linalg.solve(A,b)
print(x)

[[ 0.456]
 [ 0.274]
 [ 0.384]
 [-0.392]
 [ 0.719]]


## LDLT rozklad

LDLT rozklad je vhodný pro symetrické matice. Lze jej popsat následujícím pseudokódem:

```
n = size(A)
L = I, D = 0

v(1) = A(1, 1)
D(1, 1) = v(1)
L(2:n, 1) = A(2:n, 1)/v(1)

for j = 2 to n do
    for i = 1 to j - 1 do
        v(i) = L(j, i)D(i, i)
    end

    v(j) = A(j, j) - L(j, 1:j-1)v(1, j-1)
    D(j, j) = v(j)
    L(j+1:n, j) = (A(j+1:n, j) - L(j+1:n, 1:j-1)v(1:j-1)) / v(j)
end
```

V následující části implementujete LDLT rozklad a využijete ho k řešení soustavy se symetrickou maticí.

In [20]:
# ÚKOL: Doplňte následující kód

import numpy as np

def my_ldlt(A):
    """
    Vypočítá LDLT rozklad symetrické matice.
    Vstup: A - A symetrická matice
    Výstup: L, D - Matice dekompozice takové, že platí A = L*D*L.T
    """
    m, n = A.shape

    # Zkontrolujme, že je matice čtvercová
    if m != n:
        raise ValueError('The matrix must be square!')

    L = np.eye(m)
    D = np.zeros((m, m))

    v = np.zeros(m)
    # doplnte
    v[0] = A[0][0]
    D[0][0] = v[0]
    L[1:n][0] = A[1:n][0] / v[0]

    for j in range(1,n):
        for i in range(0,j-1):
            v[i] = L[j][i] * D[i][i]
        v[j] = A[j][j] - (L[j][0:j-1] * v[j-1])
        D[j][j] = v[j]
        L[j+1:n][j] = (A[j+1:n][j] - L[j+1:n][0:j-1]) * v[0:j-1] / v[j]

    return L, D

In [21]:
# Vygenerujeme náhodnou symetrickou matici
A = np.random.rand(5, 5)
A = A + A.transpose()
print(A)

[[1.329 0.191 1.7   1.688 0.419]
 [0.191 0.945 1.21  0.849 0.23 ]
 [1.7   1.21  0.078 0.437 0.416]
 [1.688 0.849 0.437 1.161 1.401]
 [0.419 0.23  0.416 1.401 1.445]]


In [22]:
# Otestujeme, že váš kód vrací správný výsledek
L, D = my_ldlt(A)
AA = L@D@L.transpose()
print(AA-A)

ValueError: setting an array element with a sequence.

Známe-li rozklad symetrické matice $\mathsf{A} = \mathsf{L}\mathsf{D}\mathsf{L}^T$, můžeme jej využít k řešení soustavy $\mathsf{A}\mathbf{x} = \mathbf{b}$, kterou převedeme na 
$$\mathsf{L}\mathsf{D}\mathsf{L}^T\mathbf{x} = \mathbf{b}$$
a vyřešíme vhodnými substitucemi s využitím dopředné a zpětné substituce.

Rozmyslete si, jak takové substituce budou vypadat a v následující části vyřešte náhodnou soustavu.

In [ ]:
# ÚKOL: Doplňte následující buňku a vyřešte soustavu rovnic Ax = b se symetrickou náhodnou maticí A a
# náhodným vektorem pravé strany b. Využijte vaše metody fsubst a bsubst
b = np.random.rand(5)

# doplňte řešení soustavy Ax = b pomoci LDLT

In [ ]:
# Porovnáme vaše řešení a řešen pomocí numpy
print(vase_x)
print(np.linalg.solve(A, b))